In [1]:
## Import the libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')




### Check for Headers and Footers

In [2]:
# load the Raw data
# Given Data

#set low_memory to False - avoid dtype error
dt = pd.read_csv('Loan.csv', low_memory=False)
#dt= pd.read_csv("loan.csv")

#check for headers 
dt.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Loan.csv'

In [ ]:
#check for footers
dt.tail()

#The data set has no headers or footers 

In [ ]:
# To get an insight of the RAW data
dt.info()
dt.shape

#There are 39717 rows and 111 columns -(39717, 111)

### Data Preparation

In [ ]:
#-Data Cleaning

#To find out the total number of columns whose values are all NaN

null_columns = dt.columns[dt.isnull().all()]
print(len(null_columns))



In [ ]:
# Out of 111 columns, we have 54 columns whose values are all NaN

# We can remove these 54 columns as these columns can not contribute anything for our analysis


In [ ]:
#Removing the columns whose values are all NaN
dt.dropna(axis = 1, how = 'all', inplace = True)

dt.shape



In [ ]:
# We have successfully dropped 54 null valued columns from the data set

In [ ]:
#To get columns which has single/same value throughout
same_valued_columns = dt.columns[dt.nunique() == 1]

# Print the number of single-valued columns
print(len(same_valued_columns))
print(same_valued_columns)



In [ ]:
# There are 9 columns which has same value throughout
# As these 9 column value does not change with other variables, it will have no impact in our data analysis
# We can remove these 9 columns from the data set


In [ ]:
dt.drop(['pymnt_plan', "initial_list_status",'collections_12_mths_ex_med','policy_code','acc_now_delinq', 'application_type', 'pub_rec_bankruptcies', 'tax_liens', 'delinq_amnt'], axis = 1, inplace = True)
dt.shape

In [ ]:
#To get columns which has all value unique throughout
Allunique_valued_columns = dt.columns[dt.nunique() == 39717]

# Print the number of single-valued columns
print(len(Allunique_valued_columns))
print(Allunique_valued_columns)

In [ ]:
# dropping the column 'id', 'member_id',and 'url'

dt.drop(['id', 'member_id', 'url'],axis = 1, inplace = True)



In [ ]:
dt.shape


In [ ]:
# The 45 columns we are left with after cleaning 
dt.columns



In [ ]:
# We have only 45 columns after doing the basic clean of NaN and constant valued columns

# Dropping off irrelevant column for our analysis from the defintion referring the Dictionary excel file

# Dropping off Customer Behaviours Variables

dt.drop(['funded_amnt','funded_amnt_inv','inq_last_6mths','total_acc','open_acc','revol_util','pub_rec','sub_grade', 'emp_title','desc','title','zip_code','out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'chargeoff_within_12_mths', 'addr_state','revol_bal','delinq_2yrs', 
        'earliest_cr_line','mths_since_last_delinq', 'mths_since_last_record', ],axis = 1, inplace = True)

In [ ]:
dt.shape

### Relevant Colums for our Analysis

In [ ]:
#After dropping the consumer behaviourial variables and irrelevant columns

#Now, we have only 13 columns relevant columns

dt.columns


In [ ]:
#Finding the null values in the current data set

null_percentage = dt.isnull().sum() / len(dt) * 100
print(null_percentage)

In [ ]:
# We have 2.7% null values in emp_length

# undersatnding the distribution of emp_length

dt.emp_length.value_counts()

In [ ]:
# Mode of emp_length

print("Mode : " + dt.emp_length.mode()[0])

In [ ]:
#The mode 10+years value has far higher frequency than that of the next most frequent value.

#Hence assign the mode values for NaN , as it will be safe

dt.emp_length.fillna(dt.emp_length.mode()[0], inplace = True)
dt.emp_length.isna().sum()

In [ ]:
# We have succesfully handled the missing value of the data set

In [ ]:
#On Observig the data set ,the records where the loan_status ='Current' will not be useful for our analysis

#Now we will restric the data only to loan status=charged off and Loan status=Fully Paid

# We will remove loan_status='Current' variables in the current data set

dt=dt[dt['loan_status']!='Current']
dt.shape

In [ ]:
# Understanding the data types of current data set
dt.dtypes

In [ ]:
# Understanding the values of the current data set

#dt['loan_amnt'].value_counts()
#dt['int_rate'].value_counts()
#dt['term'].value_counts()
#dt['installment'].value_counts()
#dt['emp_length'].value_counts()
#dt['home_ownership'].value_counts()
#dt['loan_status'].value_counts()
#dt['annual_inc'].value_counts()
#dt['purpose'].value_counts()
#dt['verification_status'].value_counts()
#dt['issue_d'].value_counts()
#dt['dti'].value_counts()



### Data Standardization

### After observing the current data set ,we found that few variables needed data standardizing to use them in our analysis

### Varaibles need to be standardized are int_rate,issue_d



### Variable term has only two values 30 months and 60 months we will keep them as it is without changing it to numeric

### Similarly we will keep emp_length as a categorical variable , as 10+ years can be any practical number


In [ ]:
#Standardizing the interest rate column 
#Strip out '%' and convert the value to float for numerical analysis
dt['int_rate']=dt['int_rate'].str.strip('%').astype(float)
dt['int_rate'].dtype
dt['int_rate'].value_counts()

In [ ]:

# Removing the phrase years','year'and blank spaces from emp_length for better readability and visualization

def std_emp_length(emp_length):
    
    emp_length=emp_length.replace("years","")
    emp_length=emp_length.replace("year","")
    #emp_length=emp_length.replace("<1","0")
    #emp_length=emp_length.replace("+","")
    emp_length=emp_length.replace(" ","")
    return emp_length
dt['emp_length']=dt['emp_length'].astype(str).apply(lambda x: std_emp_length(x))
dt['emp_length'].value_counts()
    

    

In [ ]:

#Standardizing the variable "issue_d" by converting and formatting the object to date 

dt['issue_d'] = pd.to_datetime(dt['issue_d'], format='%b-%y')
dt['issue_d'].dtype

# Deriving two new separate variables for month and year from 'issue_d'

dt['issue_month'] = dt['issue_d'].dt.strftime('%b') # for better representaion of month as Jan,Feb.. instead of 1,2..
dt['issue_year'] = dt['issue_d'].dt.year

In [ ]:
dt['issue_month'].value_counts() #understanding the value distribution of issue_month

In [ ]:
dt['issue_year'].value_counts() #understanding the value distribution of issue_year

In [ ]:
dt['home_ownership'].value_counts() #understanding the value distribution of home_Ownership

In [ ]:
# Imputing the value for home_ownership variable where value is NONE and assume that to OTHER, 

# As we have only 3 data having value 'NONE', it is safe to change to OTHER

# Moreover,the term "OTHER" and "NONE" both has same context in analysis point of view

dt['home_ownership']=dt['home_ownership'].str.replace('NONE','OTHER')
dt['home_ownership'].value_counts()

## Outlier Treatment

In [ ]:
# Now we have cleaned and standardized the data set, we can find the outliers to get the most relevant data for analysis

# Handling Outliers for numeric variables

# Handling Outlier for loan_amnt

print(dt.loan_amnt.quantile([0.25,0.5, 0.75,0.90, 0.95, 0.97,0.98, 0.99]))

sns.boxplot(x=dt['loan_amnt'])
plt.show()



In [ ]:
# From the above representation, it is evident that there are outliers for loan_amnt

# From the quantile information ,we can observe the values are continuous until 0.98 and there is sudden spike,

# Hence considering the value only inside the quantile 0.98


loan_amnt_Q98 = dt['loan_amnt'].quantile(0.98)
dt = dt[dt.loan_amnt <= loan_amnt_Q98]

sns.boxplot(x=dt['loan_amnt'])
plt.show()

# The loan_amnt visualization after removing the outliers shown below



In [ ]:
# Handling outliers for dti variable

print(dt.dti.quantile([0.5, 0.75,0.90, 0.95, 0.97,0.98, 0.99]))

sns.boxplot(x=dt['dti'])
plt.show()



In [ ]:
# There are no outliers identified with dti, hence keeping the dti variable data as it is.

In [ ]:
# Handling outliers for annual_inc variable

print(dt.annual_inc.quantile([0.25,0.5, 0.75,0.90, 0.95, 0.97,0.98, 0.99]))

sns.boxplot(x=dt['annual_inc'])
plt.show()

In [ ]:
# From the above representation ,There are clearly outliers in annual_inc

# The value after 0.95 quantile seems to be disconnected with the distribution

# Hence ,We will restrict our data set to 0.95 quantile to avoid the irrelevant outlier data  



annual_inc_Q95 = dt['annual_inc'].quantile(0.95)
dt = dt[dt.annual_inc <= annual_inc_Q95]

sns.boxplot(x=dt['annual_inc'])
plt.show()



## Data Analysis

### Visualization of Target State Variable

In [ ]:
#Target state visualiazation with the current data

#The current ratio between Fully Paid and Charged off is approx 6:1 

#Currently for every 6 applicant one applicant is a defaulter



dt['loan_status'].value_counts()

In [ ]:
# Now we a clean and more relevant data in our data set for analysis

# We will visualize each variable against the target state "Loan Status -Charged off"

#Current status intrepretation of the loan application from our data set 

sns.countplot(x ='loan_status', data = dt)

### We have the clean , standardized and relevant data in our data set 

###  We will do the EDA on our data to determine the strong factors for loan defaults




## Univariate Analysis :

### Analysis of each available variable against the loan status where status is charged off

In [ ]:
# Analysing Grade (we are limiting oyr analysis at (grade) Grade Level not considering sub-grade)

sns.countplot(x = 'grade', data = dt[dt.loan_status == 'Charged Off'], order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'])

In [ ]:
# Analysing the variable issued_year (derived variable from issue_d)

sns.countplot(x='issue_year', data=dt[dt['loan_status']=='Charged Off'])

In [ ]:
# Analysing the variable issued_month (derived variable from issue_d)

sns.countplot(x='issue_month', data=dt[dt['loan_status']=='Charged Off'],order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])



In [ ]:
# Analysing the variable term 

sns.countplot(x='term', data=dt[dt['loan_status']=='Charged Off'])

In [ ]:
# Analysing the variable emp_length 

sns.countplot(x='emp_length', data=dt[dt['loan_status']=='Charged Off'],order = ['<1','1', '2', '3', '4', '5', '6', '7', '8', '9','10+'])

In [ ]:
# Analysing the variable home_ownership 
sns.countplot(x='home_ownership', data=dt[dt['loan_status']=='Charged Off'])

In [ ]:
# Analysing the variable verification_status 
sns.countplot(x='verification_status', data=dt[dt['loan_status']=='Charged Off'])

In [ ]:
# Analysing the variable purpose 
sns.countplot(y='purpose', data=dt[dt['loan_status']=='Charged Off'])

In [ ]:
dt.describe() #TO understand the data range and to determine the bin range for numerical variables

In [ ]:
# creating bins for better understanding of the distribution and visualize the numerical distribution as categorical

# after understanding the data distribution bins are created for the following variables

# installment,loan_amnt,dti,annual_inc,int_rate

dt['installment_category'] = pd.cut(dt['installment'], bins=6,precision =0,labels=['0-185','185-370','370-555','555-740','740-925','925-1110'])
dt['loan_amnt_category'] = pd.cut(dt['loan_amnt'], bins=5,precision =0,labels=['0-6k','6k-12k','12k-18k','18k-24k','24k-30k'])
dt['dti_category'] = pd.cut(dt['dti'], bins=5,precision =0,labels=['0-6','6-12','12-18','18-24','24-30'])
dt['int_rate_category'] = pd.cut(dt['int_rate'], bins=5,precision =0,labels=['5%-9%','9%-13%','13%-17%','17%-21%','21%-24%'])
dt['annual_inc_category'] = pd.cut(dt['annual_inc'], bins=5,precision =0,labels =['0k-28k','28k-56k','56k-84k','84k-112k','112k-140k'])


In [ ]:
# Analysing the variable loan_amnt 
sns.countplot(x='loan_amnt_category', data=dt[dt['loan_status']=='Charged Off'])

In [ ]:
# Analysing the variable int_rate 
sns.countplot(x='int_rate_category', data=dt[dt['loan_status']=='Charged Off'])

In [ ]:
# Analysing the variable installment 

sns.countplot(x='installment_category', data=dt[dt['loan_status']=='Charged Off'])

In [ ]:
# Analysing the variable dti 

sns.countplot(x='dti_category', data=dt[dt['loan_status']=='Charged Off'])

In [ ]:
# Analysing the variable annual Income 

sns.countplot(x='annual_inc_category', data=dt[dt['loan_status']=='Charged Off'])

## Understanding the correlation between numerical variables

In [ ]:
# Correalation between the numeric value

numeric_columns = ['loan_amnt','int_rate','installment','annual_inc','dti']
correlation_matrix = dt[numeric_columns].corr()
f,ax = plt.subplots(figsize= (10,5))

sns.heatmap(correlation_matrix,xticklabels = correlation_matrix.columns.values,yticklabels= correlation_matrix.columns.values,
           cmap= 'coolwarm', annot = True)
plt.xticks(rotation=50)
plt.show()

In [ ]:
# we will consider loan_amnt for our analysis as funded_amnt and funded_amnt_inv has similar distribution as loan_amnt 

# Analysing loan_amnt with other columns for more insights



## Bivariate Analysis :

In [ ]:
# Bivariate Analysis

# Loan amount vs Pursose over loan_status

plt.figure(figsize=(12,9))
sns.barplot(x='loan_amnt', y='purpose', hue ='loan_status',data=dt,ci=None, palette="CMRmap",saturation=1)
plt.show()

In [ ]:
# Loan amount vs term over loan_status

plt.figure(figsize=(5,8))
sns.barplot(y='loan_amnt', x='term', hue ='loan_status',data=dt,palette="CMRmap",saturation=1)
plt.show()

In [ ]:
# Loan amount vs interesr rate over loan_status

plt.figure(figsize=(9,6))
sns.barplot(y='loan_amnt', x='int_rate_category', hue ='loan_status',data=dt,palette="CMRmap",saturation=1)
plt.show()

In [ ]:
# Loan amount vs installments over loan_status

plt.figure(figsize=(9,6))
sns.barplot(y='loan_amnt', x='installment_category', hue ='loan_status',data=dt,palette="CMRmap",saturation=1)
plt.show()

In [ ]:
# Loan amount vs Grade over loan_status

plt.figure(figsize=(9,6))
sns.barplot(y='loan_amnt', x='grade', hue ='loan_status',data=dt,palette="CMRmap",order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'],saturation=1)
plt.show()

In [ ]:
# Loan amount vs Employee Length over loan_status

plt.figure(figsize=(9,6))
sns.barplot(y='loan_amnt', x='emp_length', hue ='loan_status',data =dt,order = ['<1','1', '2', '3', '4', '5', '6', '7', '8', '9','10+'],palette="CMRmap",saturation=1)
plt.show()

In [ ]:
# Loan amount vs Home Ownership  over loan_status


plt.figure(figsize=(9,5))
sns.barplot(y='loan_amnt', x='home_ownership', hue ='loan_status',data =dt,palette="CMRmap",saturation=1)

plt.show()

In [ ]:
# Loan amount vs annual_income over loan_status

plt.figure(figsize=(9,6))
sns.barplot(y='loan_amnt', x='annual_inc_category', hue ='loan_status',data =dt,palette="CMRmap",saturation=1)

plt.show()

In [ ]:
# Loan amount vs Verification Status over loan_status

plt.figure(figsize=(9,6))
sns.barplot(y='loan_amnt', x='verification_status', hue ='loan_status',data =dt,palette="CMRmap",saturation=1)

plt.show()

In [ ]:
# Loan amount vs Debt to Income  over loan_status

plt.figure(figsize=(9,6))
sns.barplot(y='loan_amnt', x='dti_category', hue ='loan_status',ci=None,data =dt,palette="CMRmap",saturation=1)
plt.show()

In [ ]:
# Loan amount vs issue month over loan_status

plt.figure(figsize=(9,6))
sns.barplot(y='loan_amnt', x='issue_month', hue ='loan_status',data =dt,palette="CMRmap",order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],saturation=1)

plt.show()

In [ ]:
# Analysing annual income with other columns for more insights

# Annual income vs loan purpose

plt.figure(figsize=(9,9))
sns.barplot(x='annual_inc', y='purpose', hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Annual income vs Term 

plt.figure(figsize=(9,4))
sns.barplot(x='annual_inc', y='term', hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Annual income vs Grade

plt.figure(figsize=(9,6))
sns.barplot(y='annual_inc', x='grade', hue ='loan_status',data =dt,order = ['A','B', 'C', 'D', 'E', 'F', 'G'],palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Annual income vs Employee Length 

plt.figure(figsize=(9,6))
sns.barplot(y='annual_inc', x='emp_length', hue ='loan_status',data =dt,order = ['<1','1', '2', '3', '4', '5', '6', '7', '8', '9','10+'],palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Annual income vs Home Ownership

plt.figure(figsize=(9,6))
sns.barplot(y='annual_inc', x='home_ownership', hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Annual income vs Debt to Income

plt.figure(figsize=(9,6))
sns.barplot(y='annual_inc', x='dti_category', hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Annual income vs issue month

plt.figure(figsize=(9,6))
sns.barplot(y='annual_inc', x='issue_month', hue ='loan_status',data =dt,palette="colorblind",ci=None,order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],saturation=1)
plt.show()

In [ ]:
# Annual income vs installment

plt.figure(figsize=(9,6))
sns.barplot(y='annual_inc', x='installment_category', hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Annual income vs Loan Amount

plt.figure(figsize=(9,6))
sns.barplot(y='annual_inc', x='loan_amnt_category', hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Annual income vs Iterest Rate

plt.figure(figsize=(9,6))
sns.barplot(y='annual_inc', x='int_rate_category', hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Analysing interst rate with other columns for more insights

# Bivariate Analysis


In [ ]:
# Interest Rate and loan Amount

plt.figure(figsize=(9,4))
sns.barplot(x='int_rate', y='loan_amnt_category', hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Interest Rate and  Annual Income

plt.figure(figsize=(9,4))
sns.barplot(x='int_rate', y='annual_inc_category', hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Interest Rate and Debt To Incoome

plt.figure(figsize=(9,4))
sns.barplot(x='int_rate', y='dti_category', hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Interest Rate and Instalment

plt.figure(figsize=(9,4))
sns.barplot(x='int_rate', y='installment_category', hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Interest Rate and Grade

plt.figure(figsize=(9,4))
sns.barplot(x='int_rate', y='grade', order=['A','B','C','D','E','F','G'],hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Interest Rate and Employee Length

plt.figure(figsize=(9,4))
sns.barplot(x='int_rate', y='emp_length',order=['<1','1','2','3','4','5','6','7','8','9','10+'], hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Interest Rate and Verfication Status

plt.figure(figsize=(9,4))
sns.barplot(x='int_rate', y='verification_status', hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Interest Rate and Home Ownership

plt.figure(figsize=(9,4))
sns.barplot(x='int_rate', y='home_ownership', hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

In [ ]:
# Interest Rate and Loan Purpose

plt.figure(figsize=(9,7))
sns.barplot(x='int_rate', y='purpose', hue ='loan_status',data =dt,palette="colorblind",ci=None,saturation=1)
plt.show()

## Listed below are the Observation from Univariate Analysis

In [ ]:
# 1.	Applicants with grade B are high in charged off status
# 2.	More charged off occurred in the year 2011 
# 3.	More charged off occurred in the month of December
# 4.	Term with 36 months is charged off in high numbers
# 5.	Employment length more than 10 years charged off in high numbers, this could be a fact that all the applicants above 10 years are in the category 10+ years 
# 6.	Applicant who is residing in rented home are charged off in high numbers
# 7.	Applicants whose source are not verified are charged off in high numbers
# 8.	Applicants used loan for debt consolidation are charged off in high numbers
# 9.	Applicants with loan between 500 to 12000 are charged off in high numbers
# 10.	Applicants with interest rate between 13 and 17 are charged off in high numbers
# 11.	Applicants with instalments between 185 and 370 are charged off in high numbers
# 12.	Applicants with debt-to-income ratio between 12 and 18 are charged off in high numbers
# 13.	Applicants with annual income between 28k and 56K are charged off in high numbers


## Listed below are the Observation from Bivariate Analysis

In [ ]:
# 1.	If Loan Amount is higher than 10K for the purpose small business, debt consolidation, credit card 
# 2.	If Loan amount more than 12K, and term is 60 months
# 3.	Loan amount higher than15K and interest rate is higher than 20%
# 4.	Loan amount higher than 25K and instalment is higher than 925
# 5.	Loan amount higher than 15K and Grade of the applicants are F and G
# 6.	Applicants whose Home is in Mortgage
# 7.	Loan amount is higher and even if the verification is done
# 8.	Applicant’s Annual income is higher than 60k with purpose of loan is for high home improvement, small business
# 9.	Applicant’s Annual income is higher than 60k and falls in lower Grade as F and G 
# 10.	Interest Rate is higher than 14% and purpose of loan is small business



### End of Analysis

### Contributors: 

   ### 1.Antony John Sundar Aruldos
   ### 2.Subhrabindu Khuntia